### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2022 Semester 1

## Assignment 1: Naive Bayes Leaner for Adult Database


**Student Name(s):** `Jie Xie`
<br>
**Student ID(s):** `1174437`



Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

## General info

<b>Due date</b>: Friday, 8 April 2022 7pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: This iPython notebook is a template which you will use for your Assignment 1 submission. You need to only submitted the completed copy of this iPython notebook.

<b>Late submissions</b>: -10% per day up to 5 days (both weekdays and weekends count). Submissions more than 5 days late will not be accepted (resul in a mark of 0).
<ul>
    <li>one day late, -1.0;</li>
    <li>two days late, -2.0;</li>
    <li>three days late, -3.0;</li>
    <li>four days late, -4.0;</li>
    <li>five days late, -5.0;</li>
</ul>

<b>Extensions</b>: Students who are demonstrably unable to submit a full solution in time due to medical reasons or other trauma, may apply for an extension.  In these cases, you should email <a href="mailto:ni.ding@unimelb.edu.au">Ni Ding</a> as soon as possible after those circumstances arise. If you attend a GP or other health care service as a result of illness, be sure to provide a Health Professional Report (HPR) form (get it from the Special Consideration section of the Student Portal), you will need this form to be filled out if your illness develops into something that later requires a Special Consideration application to be lodged. You should scan the HPR form and send it with the extension requests.

<b>Marks</b>: This assignment will be marked out of 20, and make up 20% of your overall mark for this subject.

<b>Materials</b>: Use Jupyter Notebook and Python page on Canvas for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn. You can use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it. We reserve the right to deduct up to 2 marks for unreadable or exessively inefficient code.

8 of the marks available for this Project will be assigned to whether the four specified Python functions work in a manner consistent with the materials from COMP30027. Any other implementation will not be directly assessed (except insofar as it is required to make these five functions work correctly).

12 of the marks will be assigned to your responses to the questions, in terms of both accuracy and insightfulness. We will be looking for evidence that you have an implementation that allows you to explore the problem, but also that you have thought deeply about the data and the behaviour of the Naive Bayes classifier.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board (ED -> Assignments -> A1); we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/124196/modules#module_662096">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

**IMPORTANT**

Please carefully read and fill out the <b>Authorship Declaration</b> form at the bottom of the page. Failure to fill out this form results in the following deductions: 
<UL TYPE=”square”>
<LI>missing Authorship Declaration at the bottom of the page, -5.0
<LI>incomplete or unsigned Authorship Declaration at the bottom of the page, -3.0
</UL>
**NOTE: COMPLETE AND SUBMIT THIS FILE. YOU SHOULD IMPLEMENT FOUR FUNCTIONS AND INCLUDE YOUR ANSWERS TO THE QUESTIONS IN THIS FILE ONLY. NO OTHER SUBMISSION IS REQUIRED.**

**Keep your code clean. Adding proper comments to your code is MANDATORY.**

## Part 1: Base code [8 marks]

Instructions
1. Do **not** shuffle the data set
2. Treat the attributes as they are(e.g., do **not** convert numeric attributes to categorical or categorical to numeric). Implement a Naive Bayes classifier with appropriate likelihood function for each attribute.
3. You should implement the Naive Bayes classifier from scratch. Do **not** use existing implementations/learning algorithms.
4. You CANNOT have more than one train or predict function. Both continuous numeric attributes and categorical ones should be trained in one `train()` function, similarly for the `predict()`.  
5. Apart from the instructions in point 3, you may use libraries to help you with data reading, representation, maths or evaluation
6. Ensure that all and only required information is printed, as indicated in the final three code cells. Failure to adhere to print the required information will result in **[-1 mark]** per case. *(We don't mind details like you print a list or several numbers -- just make sure the information is displayed so that it's easily accessible)
7. You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 
8. You should add adequate comments to make your code easily comprehendible.*

In [1]:
import numpy as np
import pandas as pd
import math
import scipy.stats

In [2]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
# and implement 90-10 splitting as specified in the project description.
def preprocess(filename):
    # read the data from a file
    df = pd.read_csv(filename)
    
    # calculate the length of the data before splitting 
    df_len = len(df)
    
    # define split ratio
    split_ratio = 0.9
    
    # implement 90-10 splitting
    X_train = df.iloc[ :int(split_ratio * df_len), :-1]
    X_test = df.iloc[int(split_ratio * df_len): , :-1]
    y_train = df.iloc[ :int(split_ratio * df_len), -1:]
    y_test = df.iloc[int(split_ratio * df_len): , -1:]
    
    return X_train, X_test, y_train, y_test

In [3]:
# This function divides the attributes into nominal or numeric
def divide_attributes(df):
    num_attribute = len(df.columns)
    nominal_attributes = []
    numeric_attributes = []
    for i in range(num_attribute):
        if (df.dtypes[i] == 'int64'):
            numeric_attributes.append(df.columns[i])
        else:
            nominal_attributes.append(df.columns[i])
    return numeric_attributes, nominal_attributes

In [4]:
# This function calculates the log-based prior probability of one label, logP(cj)
def log_prior_prob(label, y_train):
    count = 0
    for i in range(len(y_train)):
        if (label == y_train['label'].iloc[i]):
            count += 1
    log_prior_prob = np.log(count/len(y_train))
    return log_prior_prob

In [5]:
# This function calculates the log-based likelihood for numeric attributes, P(Xi|cj)
def log_gaussian_pdf(mean, sd, x):
    return np.log((1 / sd * math.sqrt(2 * math.pi)) * np.exp(-(x - mean) ** 2 / (2 * sd ** 2)))

In [6]:
def mean_sd_dataframe(X_train, y_train, numeric_attributes):
    c1_mean_list = []
    c1_sd_list = []
    c2_mean_list = []
    c2_sd_list = []
    parameter_list = []
    
    # get indexes of the instances with label "<=50K"
    c1_index = y_train.loc[y_train['label']==' <=50K'].index
    # get indexes of the instances with label ">50K"
    c2_index = y_train.loc[y_train['label']==' >50K'].index
    
    for attribute in numeric_attributes:
        c1_attribute = X_train.loc[c1_index, attribute]
        c1_mean = np.mean(c1_attribute)
        c1_sd = np.std(c1_attribute)
        
        c2_attribute = X_train.loc[c2_index, attribute]
        c2_mean = np.mean(c2_attribute)
        c2_sd = np.std(c2_attribute)
        
        c1_mean_list.append(c1_mean)
        c1_sd_list.append(c1_sd)
        c2_mean_list.append(c2_mean)
        c2_sd_list.append(c2_sd)
        
    parameter_list.append(c1_mean_list)
    parameter_list.append(c1_sd_list)
    parameter_list.append(c2_mean_list)
    parameter_list.append(c2_sd_list)
    
    df = pd.DataFrame(parameter_list,
                     index = ['c1_mean', 'c1_sd', 'c2_mean', 'c2_sd'],
                     columns = numeric_attributes)
    
    return df

In [7]:
def get_likelihood(X_train, y_train, nominal_attributes):
    parameter_list = []
    
    # get indexes of the instances with label "<=50K"
    c1_index = y_train.loc[y_train['label']==' <=50K'].index
    # get indexes of the instances with label ">50K"
    c2_index = y_train.loc[y_train['label']==' >50K'].index
    
    # attempt
    train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
    
    for attribute in nominal_attributes:
        # get all the attribute values of a specific attribute
        attribute_value = list(set(X_train[attribute]))
        
        c1_attribute = pd.DataFrame(X_train.loc[c1_index, attribute])
        c2_attribute = pd.DataFrame(X_train.loc[c2_index, attribute])
        
        c1_lld_list = []
        c2_lld_list = []
        
        for value in attribute_value:
            
            # calculate the log likelihood of a nominal attribute with label c1
            # c1_likelihood = np.log(len(y_train.loc[y_train['label']==label])/len(c1_index))
            if train[train['label']==' <=50K'].shape[0] == 0 or train[(train[attribute]==value)&(train['label']==' <=50K')].shape[0] == 0:
                c1_likelihood = 0
            else:
                c1_likelihood = np.log(train[(train[attribute]==value)&(train['label']==' <=50K')].shape[0] / train[train['label']==' <=50K'].shape[0]) 
            
            
            # calculate the log likelihood of a nominal attribute with label c2
            # c2_likelihood = np.log(len(y_train.loc[y_train['label']==label])/len(c2_index))
            if train[train['label']==' >50K'].shape[0] == 0 or train[(train[attribute]==value)&(train['label']==' >50K')].shape[0] == 0:
                c2_likelihood = 0
            else:
                c2_likelihood = np.log(train[(train[attribute]==value)&(train['label']==' >50K')].shape[0] / train[train['label']==' >50K'].shape[0])
                        
            
            c1_lld_list.append(c1_likelihood)
            c2_lld_list.append(c2_likelihood)
            
        parameter_list.append((c1_lld_list, c2_lld_list))
        
    # outout the array of DataFrame
    output = []
    for i in range(len(parameter_list)):
        df = pd.DataFrame([parameter_list[i][0], parameter_list[i][1]],
                         columns = [list(set(X_train[nominal_attributes[i]]))])
        output.append(df)
    return output

In [8]:
# This function should calculat prior probabilities and likelihoods (conditional probabilities) from the training data and using
# to build a naive Bayes model
def train(X_train, y_train):
    
    # calculate the log prior probabilities
    prior_c1 = log_prior_prob(' <=50K', y_train)
    prior_c2 = log_prior_prob(' >50K', y_train)
    prior = pd.DataFrame([prior_c1, prior_c2], columns=['log_prior'])
    
    # seperate numeric and nominal attributes
    numeric_attributes, nominal_attributes = divide_attributes(X_train)
    
    # store the mean and sd for each numeric attributes
    numeric_parameters = mean_sd_dataframe(X_train, y_train, numeric_attributes)
    
    # store the likelihoods for each nominal attributes
    nominal_parameters = get_likelihood(X_train, y_train, nominal_attributes)
    
    return prior, numeric_parameters, nominal_parameters

In [9]:
def seperate_index(df):
    num_attribute = len(df.columns)
    nominal_index = []
    numeric_index = []
    
    for i in range(num_attribute):
        if (df.dtypes[i]=='int64'):
            numeric_index.append(i)
        else:
            nominal_index.append(i)
    
    return numeric_index, nominal_index

In [10]:
def sum_gaussian_pdf(numeric_attr, numeric_parameters):
    c1_lld = 0
    c2_lld = 0
    
    for i in range(len(numeric_attr)):
        x = numeric_attr[i]
        # calculate c1 lld
        mean1 = numeric_parameters[numeric_attr.index[i]][0]
        sd1 = numeric_parameters[numeric_attr.index[i]][1]
        # c1_lld += log_gaussian_pdf(mean1, sd1, x)
        log_pdf1 = log_gaussian_pdf(mean1, sd1, x)
        if log_pdf1 is not None:
            c1_lld = c1_lld + log_pdf1
        else:
            c1_lld += 0
        
        # calculate c2 lld
        mean2 = numeric_parameters[numeric_attr.index[i]][2]
        sd2 = numeric_parameters[numeric_attr.index[i]][3]
        # c2_lld += log_gaussian_pdf(mean2, sd2, x)
        log_pdf2 = log_gaussian_pdf(mean2, sd2, x)
        if log_pdf2 is not None:
            c2_lld = c2_lld + log_pdf2
        else:
            c2_lld += 0
                
    return c1_lld, c2_lld

In [11]:
def sum_likelihood(nominal_attr, nominal_parameters):
    c1_lld = 0
    c2_lld = 0
    
    for i in range(len(nominal_attr)):
        x = nominal_attr[i]        
        for j in range(len(nominal_parameters)):
            if x in nominal_parameters[j]:
                curr_df = nominal_parameters[j]
                
                # calculate c1 lld
                c1_lld = curr_df.iloc[0][x]
                # calculate c2 lld
                c2_lld = curr_df.iloc[1][x]
    
    return c1_lld, c2_lld

In [12]:
# This function should predict classes for new items in the testing data
def predict(X_test, prior, numeric_parameters, nominal_parameters):
    predicted_label = []
    c1_hat = []
    c2_hat = []
    
    numeric_index, nominal_index = seperate_index(X_test)
    c1_prior = prior['log_prior'][0]
    c2_prior = prior['log_prior'][1]
    
    for i in range(len(X_test)):
        c1_numeric_lld, c2_numeric_lld = sum_gaussian_pdf(X_test.iloc[i, numeric_index], numeric_parameters)
        c1_nominal_lld, c2_nominal_lld = sum_likelihood(X_test.iloc[i, nominal_index], nominal_parameters)
        
        # calculate c1 total log posterior
        c1_posterior = c1_prior + c1_numeric_lld + c1_nominal_lld
        c1_hat.append(c1_posterior)
        # calculate c2 total log posterior
        c2_posterior = c2_prior + c2_numeric_lld + c2_nominal_lld
        c2_hat.append(c2_posterior)
        
        # compare c1_posterior and c2_posterior
        if c1_posterior > c2_posterior:
            predicted_label.append(' <=50K')
        else:
            predicted_label.append(' >50K')
        
    return predicted_label, c1_hat, c2_hat

In [13]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels, return and output accuracy, confusion matrix and F1 score.
def evaluate(y_test, predicted_label):
    TP = TN = FP = FN = 0
    
    for i in range(len(y_test)):
        if (predicted_label[i]==' <=50K' and y_test['label'].iloc[i]==' <=50K'):
            TP += 1
        elif (predicted_label[i]==' >50K' and y_test['label'].iloc[i]==' >50K'):
            TN += 1
        elif (predicted_label[i]==' <=50K' and y_test['label'].iloc[i]==' >50K'):
            FP += 1
        else:
            FN += 1
    
    accuracy = (TP+TN) / (TP+TN+FP+FN)
    conf_matrix = np.array([[TP, FN], [FP, TN]])
    precision = TP / (TP+FP)
    recall = TP / (TP+FN)
    f_one = (2*precision*recall) / (precision+recall)
    
    return accuracy, conf_matrix, f_one

In [14]:
# iterate all the labels to count the number of label types
num_labels = 0
label_types = []

for i in range(y_train.shape[0]):
    if y_train.iloc[i]['label'] not in label_types:
        num_labels += 1
        label_types.append(y_train.iloc[i]['label'])
        
for i in range(y_test.shape[0]):
    if y_test.iloc[i]['label'] not in label_types:
        num_labels += 1
        label_types.append(y_test.iloc[i]['label'])

NameError: name 'y_train' is not defined

In [15]:
# This cell should act as your "main" function where you call the above functions 
# on the full ADULT data set, and print the evaluation results. [0.33 marks]


# First, read in the data and apply your NB model to the ADULT data
# Assume the adult file is stored in the dataset folder
X_train, X_test, y_train, y_test = preprocess('dataset/adult.csv')
prior, numeric_parameters, nominal_parameters = train(X_train, y_train)
predicted_label, c1_hat, c2_hat = predict(X_test, prior, numeric_parameters, nominal_parameters)
accuracy, conf_matrix, f_one = evaluate(y_test, predicted_label)


# Second, print the full evaluation results from the evaluate() function
print("Accuracy: ", accuracy, "\nConfusion Matrix: \n", conf_matrix, "\nF1 Score: ", f_one)




# Third, print data statistics and model predictions, as instructed below 
# N is the total number of instances, F the total number of attributes, L the total number of labels
# The "class probabilities" may be unnormalized
# The "predicted class ID" must be in range (0, L)

print("Attribute vectors of instances [0, 1, 2]: \n",  X_test.values[:3]) # of the first three records in adult.csv

print("\nNumber of instances (N): ", X_train.shape[0] + X_test.shape[0])
print("Number of attributes (F): ", X_test.shape[1])
print("Number of labels (L): ", num_labels)


# print out the prediction results of the last three instances
print("\n\nPredicted class log-probabilities for instance N-3: ", '<=50K:', c1_hat[X_test.shape[0]-3], '   >50K:', c2_hat[X_test.shape[0]-3])
print("Predicted class ID for instance N-3: ", predicted_label[X_test.shape[0]-3])
print("\nPredicted class log-probabilities for instance N-2: ", '<=50K:', c1_hat[X_test.shape[0]-2], '   >50K:', c2_hat[X_test.shape[0]-3])
print("Predicted class ID for instance N-2: ", predicted_label[X_test.shape[0]-2])
print("\nPredicted class log-probabilities for instance N-1: ", '<=50K:', c1_hat[X_test.shape[0]-1], '   >50K:', c2_hat[X_test.shape[0]-3])
print("Predicted class ID for instance N-1: ", predicted_label[X_test.shape[0]-1])

Accuracy:  0.79 
Confusion Matrix: 
 [[73  4]
 [17  6]] 
F1 Score:  0.874251497005988
Attribute vectors of instances [0, 1, 2]: 
 [[28 ' Private' ' HS-grad' 9 ' Married-civ-spouse' ' Tech-support'
  ' Husband' ' White' ' Male' 40 ' United-States']
 [63 ' Local-gov' ' 7th-8th' 4 ' Married-civ-spouse' ' Other-service'
  ' Husband' ' White' ' Male' 55 ' United-States']
 [51 ' Self-emp-not-inc' ' 9th' 5 ' Married-civ-spouse' ' Craft-repair'
  ' Husband' ' White' ' Male' 20 ' United-States']]

Number of instances (N):  1000
Number of attributes (F):  11
Number of labels (L):  0


Predicted class log-probabilities for instance N-3:  <=50K: -5.602452241167827    >50K: -4.761457585842188
Predicted class ID for instance N-3:   >50K

Predicted class log-probabilities for instance N-2:  <=50K: -6.294519149508661    >50K: -4.761457585842188
Predicted class ID for instance N-2:   >50K

Predicted class log-probabilities for instance N-1:  <=50K: -3.66279017367217    >50K: -4.761457585842188
Predicte

## Part 2: Conceptual questions [8 marks for groups of 1] / [16 marks for groups of 2]


If you are in a group of 1, you should respond to Q1 and Q2.

If you are in a group of 2, you should respond to Q1, Q2, Q3 and Q4.

A response to a question should take about 100–250 words. You may need to develope codes or functions to help respond to the question here. 

#### NOTE: We strongly recommend <u>including figures or tables, etc.</u> to support your responses. The figures and tables inserted in Markdown cells must be reproducable by your code.

### Q1 [4 marks]
<u>Sensitivity</u> and <u>specificity</u> are two model evaluation metrics.  A good model should have both sensitivity and specificity high. Use the $2 \times 2$ confusion matrix returned by `evaluate()` to calculate the sensitivity and specificity. Do you see a difference between them? If so, what causes this difference? Provide suggestions to improve the model performance. 

In [16]:
# Write additional code here, if necessary (you may insert additional code cells)
conf_matrix = evaluate(y_test, predicted_label)[1]
TP, FN, FP, TN = conf_matrix[0][0], conf_matrix[0][1], conf_matrix[1][0], conf_matrix[1][1]

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
print(f'Sensitivity: {sensitivity}')
print(f'Specificity: {specificity}')

Sensitivity: 0.948051948051948
Specificity: 0.2608695652173913


Provide your text answer of 150-200 words in this cell.

* The difference: Sensitivity value is significantly higher than specificity. Sensitivity is the ratio of detected positive instances out of all positive instances. In contrast, specificity is the ratio of detected negative instances out of all negative instances. Therefore we can conclude that the model has good estimation on positive instances (' <=50K'). However it does not work properly when detecting the negative instances (' >50K').


* Cause of the difference: The reason of this result might be the unbalanced data distribution. In the adult csv we can find over 75% training data have positive labels. Then the model is more likely to estimate the next instance is also positive, as the prior probability of positive label is significant greater than negative label. This can lead to high value of TP (true positive) and low value of TN (true negative). Then according to the formulas, the high value of sensitivity and low value of specificity makes sense.


* Suggestions to improve: Since a major problem is the unbalanced data distribution, we can keep the same number of positive samples and negative samples in the training data. In addition we can apply resampling technique to make sure that train and test data have the same class distribution. Despite the unbalanced data, the missing values also have potential effects to the prediction result. Therefore we can also attempt a proper method to deal with the missing values, which might enhance the model's performance.

### Q2 [4 marks]
You can adopt different methods for training and/or testing, which will produce different results in model evaluation. 

(a) Instead of Gaussian, <u>implement KDE</u> for  $P(X_i|c_j)$ for numeric attributes $X_i$. Compare the evaluation results with Gaussian. Which one do you think is more suitable to model $P(X_i|c_j)$, Gaussian or KDE? Observe all numeric attributes and justify your answer.

You can choose an arbitrary value for kernel bandwidth $\sigma$ for KDE, but a value between 3 and 15 is recommended. You should write code to implement KDE, not call an existing function/method such as `KernelDensity` from `scikit-learn`.

(b) Implement <u>10-fold and 2-fold cross-validations</u>.  
	Observe the evaluation results in each fold and the average accuracy, recall and specificity over all folds. 
	Comment on what is the effect by changing the values of $m$ in $m$-fold cross validation. (You can choose either Gaussian or KDE Naive Bayes.)

In [17]:
# This function calculates the Gaussian pdf with mean 0 and arbitrary kernel bandwidth
def gaussian_pdf(x, bandwidth, xi):
    return (1 / bandwidth * math.sqrt(2 * math.pi)) * np.exp(-(xi - x) ** 2 / (2 * bandwidth ** 2))

In [18]:
def KDE_pdf(x, c_attr, bandwidth):
    KDE_lld = []
    for i in c_attr.index:
        xi = c_attr[i]
        KDE_lld.append(gaussian_pdf(x, bandwidth, xi))
    return np.log(np.mean(KDE_lld))

In [19]:
# This function calculates the likelihood of a single-line test data corresponds to one specific class
def get_KDE_likelihood(X_train, x_test, y_train, numeric_attr, bandwidth):
    parameter_list = []    
    # get the indices of instances with label ' <=50K'
    c1_index = y_train.loc[y_train['label']==' <=50K'].index
    # get the indices of instances with label ' >50K'
    c2_index = y_train.loc[y_train['label']==' >50K'].index
    
    for attr in numeric_attr:
        c1_lld = []
        c2_lld = []
        c1_attr = X_train.loc[c1_index, attr]
        c2_attr = X_train.loc[c2_index, attr]
        
        for i in X_test.index:
            x = X_test[attr][i]
            c1_lld.append(KDE_pdf(x, c1_attr, bandwidth))
            c2_lld.append(KDE_pdf(x, c2_attr, bandwidth))
            
        parameter_list.append(c1_lld)
        parameter_list.append(c2_lld)
    
    df = pd.DataFrame(parameter_list, index=['c1_age', 'c2_age', 'c1_educ', 'c2_educ', 'c1_hours', 'c2_hours'])

    return df

In [20]:
# Write additional code here, if necessary (you may insert additional code cells)
# This function should calculat prior probabilities and likelihoods (conditional probabilities) from the training data and using
# to build a KDE naive Bayes model
def KDE_train(X_train, X_test, y_train, bandwidth):
    # calculate the log prior probabilities
    prior_c1 = log_prior_prob(' <=50K', y_train)
    prior_c2 = log_prior_prob(' >50K', y_train)
    prior = pd.DataFrame([prior_c1, prior_c2], columns=['log_prior'])
    
    # seperate numeric and nominal attributes
    numeric_attributes, nominal_attributes = divide_attributes(X_train)
    
    # store the KDE likelihood for each numeric attributes
    numeric_parameters = get_KDE_likelihood(X_train, X_test, y_train, numeric_attributes, bandwidth)
    
    # store the likelihoods for each nominal attributes
    nominal_parameters = get_likelihood(X_train, y_train, nominal_attributes)
    
    return prior, numeric_parameters, nominal_parameters

In [21]:
# Make small change on predict function to adapt KDE prediction
def KDE_predict(X_test, prior, numeric_parameters, nominal_parameters):
    predicted_label = []
    c1_hat = []
    c2_hat = []
    
    numeric_index, nominal_index = seperate_index(X_test)
    c1_prior = prior['log_prior'][0]
    c2_prior = prior['log_prior'][1]
    
    for i in range(len(X_test)):
        # obtain the data from previous dataframe
        c1_numeric_lld = (numeric_parameters[i]['c1_age'] + numeric_parameters[i]['c1_educ'] + numeric_parameters[i]['c1_hours'])
        c2_numeric_lld = (numeric_parameters[i]['c2_age'] + numeric_parameters[i]['c2_educ'] + numeric_parameters[i]['c2_hours'])
        
        c1_nominal_lld, c2_nominal_lld = sum_likelihood(X_test.iloc[i, nominal_index], nominal_parameters)
        
        # calculate c1 total log posterior
        c1_posterior = c1_prior + c1_numeric_lld + c1_nominal_lld
        c1_hat.append(c1_posterior)
        # calculate c2 total log posterior
        c2_posterior = c2_prior + c2_numeric_lld + c2_nominal_lld
        c2_hat.append(c2_posterior)
        
        # compare c1_posterior and c2_posterior
        if c1_posterior > c2_posterior:
            predicted_label.append(' <=50K')
        else:
            predicted_label.append(' >50K')
        
    return predicted_label, c1_hat, c2_hat

In [22]:
## evaluate the KDE 
X_train, X_test, y_train, y_test = preprocess('dataset/adult.csv')
prior_proba, con_proba_num, con_proba_cat = KDE_train(X_train, X_test, y_train, 10)
y_predict_kde = KDE_predict(X_test, prior_proba, con_proba_num, con_proba_cat)
acc_kde, cm_kde, f1_kde = evaluate(y_test, y_predict_kde[0])

tp_kde, fn_kde, fp_kde, tn_kde = cm_kde[0][0], cm_kde[0][1], cm_kde[1][0], cm_kde[1][1]
sensitivity_kde = tp_kde / (tp_kde + fn_kde)
specificity_kde = tn_kde / (tn_kde + fp_kde)

print("Evaluation results of KDE (bandwidth=10) ")
print("Accuracy: ", acc_kde)
print("F1: ", f1_kde)
print(f'Sensitivity: {sensitivity_kde}')
print(f'Specificity: {specificity_kde}')

Evaluation results of KDE (bandwidth=10) 
Accuracy:  0.76
F1:  0.8620689655172413
Sensitivity: 0.974025974025974
Specificity: 0.043478260869565216


In [23]:
# Compare the evaluation results between Gaussian and KDE
results = pd.DataFrame(index=['Gaussian', 'KDE'], 
                       columns=['Accuracy', 'F1-Score', 'Sensitivity', 'Specificity'])
results.loc['Gaussian'] = [accuracy, f_one, sensitivity, specificity]
results.loc['KDE'] = [acc_kde, f1_kde, sensitivity_kde, specificity_kde]
results

,Accuracy,F1-Score,Sensitivity,Specificity
Gaussian,0.79,0.874251,0.948052,0.26087
KDE,0.76,0.862069,0.974026,0.043478


Provide your text answer of 150-200 words in this cell.

Q2 (a)

From the testing result, KDE looks more suitable to model 𝑃(𝑋𝑖|𝑐𝑗). Using KDE model specificity increases while the other data does not change significantly. In the perspect of numeric attributes, if their distributions follow Gaussian distribution then Gaussian naive bayes is a proper model. However if the distributions of the attributes do not match Gaussian distribution, KDE becomes a more suitable model. KDE does not assume a distribution therefore it works better for data without a clear distribution.

In [24]:
# Q2 (b)
# make some change to the evaluate function to adapt cross validation 
def CV_evaluate(y_test, predicted_label):
    TP = TN = FP = FN = 0
    
    for i in range(len(y_test)):
        if (predicted_label[i]==' <=50K' and y_test['label'].iloc[i]==' <=50K'):
            TP += 1
        elif (predicted_label[i]==' >50K' and y_test['label'].iloc[i]==' >50K'):
            TN += 1
        elif (predicted_label[i]==' <=50K' and y_test['label'].iloc[i]==' >50K'):
            FP += 1
        else:
            FN += 1
    
    accuracy = (TP+TN) / (TP+TN+FP+FN)
    recall = TP / (TP+FN)
    specificity = TN / (TN+FP)
    
    return accuracy, recall, specificity

In [25]:
def k_fold_CV(k, filename):
    accuracy_list = []
    recall_list = []
    specificity_list = []
    
    df = pd.read_csv(filename)
    split_ratio = 1/k
    split_size = split_ratio*len(df)
    
    for i in range(k):
        test_index = list(range(int(split_size*i), int(split_size*(i+1))))
        train_index = df.index.drop(test_index)
        
        X_train = df.iloc[train_index, :-1]
        X_test = df.iloc[test_index, :-1]
        y_train = df.iloc[train_index, -1:]
        y_test = df.iloc[test_index, -1:]
        
        prior, numeric_parameters, nominal_parameters = train(X_train, y_train)
        predicted_label, c1_hat, c2_hat = predict(X_test, prior, numeric_parameters, nominal_parameters)
        accuracy, recall, specificity = CV_evaluate(y_test, predicted_label)
        
        accuracy_list.append(accuracy)
        recall_list.append(recall)
        specificity_list.append(specificity)
    
    return accuracy_list, recall_list, specificity_list

In [26]:
accuracy_10_fold, recall_10_fold, specificity_10_fold = k_fold_CV(10,'dataset/adult.csv')
accuracy_10_fold, recall_10_fold, specificity_10_fold

([0.77, 0.72, 0.77, 0.79, 0.8, 0.78, 0.74, 0.78, 0.8, 0.79],
 [0.8533333333333334,
  0.8205128205128205,
  0.8831168831168831,
  0.948051948051948,
  0.9,
  0.8860759493670886,
  0.8701298701298701,
  0.8552631578947368,
  0.9315068493150684,
  0.948051948051948],
 [0.52,
  0.36363636363636365,
  0.391304347826087,
  0.2608695652173913,
  0.4,
  0.38095238095238093,
  0.30434782608695654,
  0.5416666666666666,
  0.4444444444444444,
  0.2608695652173913])

In [27]:
accuracy_2_fold, recall_2_fold, specificity_2_fold = k_fold_CV(2,'dataset/adult.csv')
accuracy_2_fold, recall_2_fold, specificity_2_fold

([0.752, 0.774],
 [0.8604651162790697, 0.9057591623036649],
 [0.3805309734513274, 0.3474576271186441])

Q2 (b)

10 fold has more reliabel result while 2 fold is more time efficient. 10 fold have more regular results, but 2 fold varies a lot. The greater the number of fold, the model uses more training data, hence it takes longer to proceed while the results are more reliable. The most extreme example would be the leave one out cross validation.

### Q3 [4 marks]
In `train()`, you are asked to treat the missing value of nominal attributes as a new category. There is another option (as suggested in Thu lecture in week 2): <u>ignoring the missing values</u>. 
Compare the two methods in both large and small datasets. Comment and explain your observations.
You can extract the first 50 records to construct a small dataset.Use Gaussian Naive Bayes only for this question.

In [6]:
# Write additional code here, if necessary (you may insert additional code cells)

Provide your text answer of 150-200 words in this cell.

### Q4 [4 marks]
In week 4, we have learned how to obtain information gain (IG) and gain ratio (GR) to choose an attribute to split a node in a decision tree. We will see how to apply them in the Naive Bayes classification.

(a) Compute the GR of each attribute $X_i$, relative to the class distribution. In the Na\"ive Bayes classifier, remove attributes in the ascending order of GR: first, remove $P(X_i|c_j)$ such that $X_i$ has the least GR; second, remove $P(X_{i'}|c_j)$ such that $X_{i'}$ has the second least GR,......, until there is only one $X_{i*}$ with the largest GR remaining in the maximand $P(c_j) P(X_{i^*} | c_j)$. Observe the <u>change of the accuracy for both Gaussian and KDE</u> (Choose bandwidth $\sigma=10$ for KDE).

(b) Compute the IG between each pair of attributes. Describe and explain your observations. Choose an attribute and implement an estimator to predict the value of `education num`. Explain why you choose this attribute. Enumerate two other examples that an attribute can be used to estimate the other and explain the reason.  

In [ ]:
# Write additional code here, if necessary (you may insert additional code cells)

### (a)

Provide your text answer to **Question 4.a** of 100-150 words in this cell.

### (b)

Provide your text answer to **Question 4.b** of 150-200 words in this cell.

<b>Authorship Declaration</b>:

   (1) I certify that the program contained in this submission is completely
   my own individual work, except where explicitly noted by comments that
   provide details otherwise.  I understand that work that has been developed
   by another student, or by me in collaboration with other students,
   or by non-students as a result of request, solicitation, or payment,
   may not be submitted for assessment in this subject.  I understand that
   submitting for assessment work developed by or in collaboration with
   other students or non-students constitutes Academic Misconduct, and
   may be penalized by mark deductions, or by other penalties determined
   via the University of Melbourne Academic Honesty Policy, as described
   at https://academicintegrity.unimelb.edu.au.

   (2) I also certify that I have not provided a copy of this work in either
   softcopy or hardcopy or any other form to any other student, and nor will
   I do so until after the marks are released. I understand that providing
   my work to other students, regardless of my intention or any undertakings
   made to me by that other student, is also Academic Misconduct.

   (3) I further understand that providing a copy of the assignment
   specification to any form of code authoring or assignment tutoring
   service, or drawing the attention of others to such services and code
   that may have been made available via such a service, may be regarded
   as Student General Misconduct (interfering with the teaching activities
   of the University and/or inciting others to commit Academic Misconduct).
   I understand that an allegation of Student General Misconduct may arise
   regardless of whether or not I personally make use of such solutions
   or sought benefit from such actions.

   <b>Signed by</b>: [Jie Xie, 1174437]
   
   <b>Dated</b>: [03/04/2022]